In [6]:
import numpy as np


### create dataset

In [3]:
# normalization
class normalization:
    
    def __init__(self, data) -> None:
        self.data = data
        
        max = np.max(data)
        self.min = np.min(data)
        
        self.range_original = max - self.min
        # range_desired = 1

    def normalize(self, data):
        new_data = (data-self.min)/self.range_original
        return new_data

    def unnormalize(self, X):
        raw_data = X*self.range_original + self.min
        return raw_data


In [4]:
# construct training data and testing data

np.random.seed(0)
X_size = (20000,2)

X = np.random.randint(1000, 9999, size=X_size)
Y = np.sum(X, axis=1)

rng = np.random.default_rng()
idx_testing = rng.choice(20000, 4000, replace=False, shuffle=False)

# testing data
X_test = X[idx_testing]
Y_test = Y[idx_testing]

# training data
mask = np.ones(Y.shape, dtype=bool)
mask[idx_testing] = False

X_train = X[mask]
Y_train = Y[mask]


NameError: name 'np' is not defined

In [5]:
# preprocess

# normalize
normalization_test = normalization(X_test)
X_test_n = normalization_test.normalize(X_test)
Y_test_n = normalization_test.normalize(Y_test)
Y_test_n = Y_test_n.reshape((-1,1))

X_train_n = normalization_test.normalize(X_train)
Y_train_n = normalization_test.normalize(Y_train)
Y_train_n = Y_train_n.reshape((-1,1))
display(X_test_n)
display(Y_test_n)


NameError: name 'X_test' is not defined

In [3]:
class shallow_nn:
    """ a shallow neural network
    dim
        input = 2
        hidden layer = n
        output = 1
    """    

    def __init__(self, n, layer_initializer, learning_rate, param_hidden, param_output=1) -> None:
        # fool-proof
        self.training = False
        
        # number of node
        self.num_input = 2
        self.num_hidden = n
        self.num_output = 1
        
        # activation function
        self.param_hidden = param_hidden
        self.param_output = param_output
        
        # hyper param
        self.learning_rate = learning_rate
        
        # init layer
        if layer_initializer is int:
            s = layer_initializer
            np.random.seed(s) # for reproducibility
            self.weights_input_hidden = np.random.uniform(size=(self.num_input, self.num_hidden))
            self.bias_hidden = np.random.uniform(size=(1, self.num_hidden))
            self.weights_hidden_output = np.random.uniform(size=(self.num_hidden, self.num_output))
            self.bias_output = np.random.uniform(size=(1, self.num_output))
            
        elif layer_initializer is list:
            ttl_layer = layer_initializer
            self.weights_input_hidden = ttl_layer[0]
            self.bias_hidden = ttl_layer[1]
            self.weights_hidden_output = ttl_layer[2]
            self.bias_output = ttl_layer[3]
        else:
            raise("missing for layer_initializer")
  
    def get_layer(self):
        ttl_layer = []
        ttl_layer.append(self.weights_input_hidden)
        ttl_layer.append(self.bias_hidden)
        ttl_layer.append(self.weights_hidden_output)
        ttl_layer.append(self.bias_output)
        return ttl_layer
  
    def act_hidden(self,x):
        """ tanh
        """
        exponient = np.e**(2*self.param_hidden*x)
        y = (exponient-1)/(exponient+1)
        return y
    
    def div_act_hidden(self,x):
        """ tanh
        """
        exponient = np.e**(self.param_hidden*x)
        exponient_minus = np.e**(-self.param_hidden*x)
        y = self.param_hidden*4/(exponient+exponient_minus)
        return y
    
    def act_output(self, x):
        """ linear
        """
        y = self.param_hidden*x
        return y
    
    def div_act_output(self, x):
        """ linear
        """
        y = self.param_hidden
        return np.array([y])
    
    def forward(self, X, training=False):
        """
        Var:
            training: bool
                True    backward later
                False   forward only
        """
        self.training = training
        if self.training is True:
            self.data = X
        else:
            self.data = None
        
        self.hidden_layer_in = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_out = self.act_hidden(self.hidden_layer_in)
        self.output_layer_in = np.dot(self.hidden_layer_out, self.weights_hidden_output) + self.bias_output
        self.output_layer_out = self.act_output(self.output_layer_in)
        
        pred = self.output_layer_out
        
        print(f"exit forward, training={training}")
        return pred
    
    def backward(self, target):
        print("enter backward")
        if self.training is False:
            print("\tif self.training is False")
        else:
            print("\tif self.training is True")
            error_output = target - self.output_layer_out
            back_output = error_output * (-1) * self.div_act_output(self.output_layer_in)
            grad_output = self.hidden_layer_out.T.dot(back_output)

            back_hidden = back_output.dot(self.weights_hidden_output.T) * self.div_act_hidden(self.hidden_layer_in)
            grad_hidden = self.data.T.dot(back_hidden)
            
            # update weight and bias
            self.weights_hidden_output -= grad_output * self.learning_rate
            self.bias_output -= np.sum(back_output, axis=0, keepdims=True) * self.learning_rate
            self.weights_input_hidden -= grad_hidden * self.learning_rate
            self.bias_hidden -= np.sum(back_hidden, axis=0, keepdims=True) * self.learning_rate

            # train 過後就關起來
            self.training = False
        print("exit backward")

    def evaluate(self, X, target):
        pred = self.forward(X)
        loss = 0.5*np.mean(np.square(target-pred))
        return loss
    
    def predict(self, X):
        pred = self.forward(X)
        return pred


In [4]:
# normalization
class normalization:
    
    def __init__(self, data) -> None:
        self.data = data
        
        max = np.max(data)
        self.min = np.min(data)
        
        self.range_original = max - self.min
        # range_desired = 1

    def normalize(self, data):
        new_data = (data-self.min)/self.range_original
        return new_data

    def unnormalize(self, X):
        raw_data = X*self.range_original + self.min
        return raw_data


##### train

In [21]:
def get_checkpoint(nn: shallow_nn, data):
    print("enter seve_checkpoint")
    x_train, y_train, x_test, y_test = data
    loss_in = nn.evaluate(x_train, y_train)
    loss_out = nn.evaluate(x_test, y_test)
    param = nn.get_layer()
    print("exit seve_checkpoint")
    return loss_in, loss_out, param 

In [26]:
def train(nn: shallow_nn, num_epoch, size_batch, data):
    x_train, y_train, _, _ = data
    ttl_loss = []
    ttl_param = []
    num_batch = len(x_train)//size_batch + 1
    rec_epoch = num_epoch//10

    loss_in, loss_out, param = get_checkpoint(nn, data)
    ttl_loss.append([loss_in, loss_out])
    ttl_param.append(param)
    for epoch in range(num_epoch):

        for batch in range(num_batch):
            idx_start = batch*num_batch
            idx_end = idx_start + size_batch
            x = x_train[idx_start:idx_end]
            y = y_train[idx_start:idx_end]
            
            # Forward pass
            nn.forward(x, training=True)
            
            # BackWard pass
            nn.backward(y)

        # compute the loss
        loss_in, loss_out, param = get_checkpoint(nn, data)
        ttl_loss.append([loss_in, loss_out])
        ttl_param.append(param)

        if epoch % rec_epoch == 0:
            print(f'Epoch {epoch} loss_in: {loss_in}, loss_out: {loss_out}')
    
    return ttl_loss, ttl_param

In [ ]:
# construct training data and testing data

np.random.seed(0)
X_size = (20000,2)

X = np.random.randint(1000, 9999, size=X_size)
Y = np.sum(X, axis=1)

rng = np.random.default_rng()
idx_testing = rng.choice(20000, 4000, replace=False, shuffle=False)

# testing data
X_test = X[idx_testing]
Y_test = Y[idx_testing]

# training data
mask = np.ones(Y.shape, dtype=bool)
mask[idx_testing] = False

X_train = X[mask]
Y_train = Y[mask]

display(X)
display(X_train)

In [ ]:
# preprocess

# normalize
normalization_test = normalization(X_test)
X_test_n = normalization_test.normalize(X_test)
Y_test_n = normalization_test.normalize(Y_test)
Y_test_n = Y_test_n.reshape((-1,1))

X_train_n = normalization_test.normalize(X_train)
Y_train_n = normalization_test.normalize(Y_train)
Y_train_n = Y_train_n.reshape((-1,1))
display(X_test_n)
display(Y_test_n)


In [27]:
nn_1 = shallow_nn(5, 1, 1, 0.0001)
data = [X_train_n, Y_train_n, X_test_n, Y_test_n]
train(nn_1, 100, 1000, data)

enter seve_checkpoint
exit forward, training=False
exit forward, training=False
exit seve_checkpoint
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True
exit backward
exit forward, training=True
enter backward
	if self.training is True


([[1.5573091574491345, 1.5604540699050111],
  [0.024241054614702207, 0.024279509986305924],
  [0.014098254239899992, 0.014145359962805226],
  [0.00777374877776576, 0.007818172458849554],
  [0.00420913800202706, 0.004245320626880977],
  [0.0023521055225751404, 0.00237878107405308],
  [0.0014383971654471663, 0.0014566036963773216],
  [0.0010055032488738005, 0.0010170616088132497],
  [0.0008043475107213922, 0.0008110463872384732],
  [0.0007106768502745202, 0.0007139693951900018],
  [0.0006655594628673491, 0.0006665223734886493],
  [0.0006419686356652394, 0.0006413606771449032],
  [0.000627792151695219, 0.0006261331001614066],
  [0.0006176938637960015, 0.0006153341149710459],
  [0.0006093573854543287, 0.000606531071930531],
  [0.000601792963321609, 0.0005986558228811593],
  [0.0005945863291225376, 0.0005912417553262798],
  [0.0005875693599473651, 0.0005840859778039185],
  [0.0005806771984979722, 0.0005771005972489891],
  [0.000573887010071323, 0.0005702475317273045],
  [0.00056719217009491